In [1]:
import numpy as np

# `site-analysis` draft documentation

`site_analysis` is a Python module for analysing molucular dynamics trajectories to identify the movement of mobile atoms through sequences of &ldquo;sites&rdquo;.

The central objects in `site_analysis` are the set of mobile atoms, represented by `Atom` objects, and the set of sites, represented by `Site` objects. `Site` objects are subclassed according to how site occupations are defined; you can work with `PolyhedralSite`, `SphericalSite`, and `VoronoiSite` types.

## example trajectory analysis

For this example we want to analyse a simulation trajectory using polyhedral sites. An example structure to analyse is defined in the file `POSCAR`, which we read in as a pymatgen `Structure`:


In [2]:
from pymatgen.io.vasp import Poscar
structure = Poscar.from_file('POSCAR').structure
print(structure.composition)

Na88 Sn16 P8 S96


Our mobile atoms are Na. These are tracked using their index in the structures (`site_analysis` assumes that atom order is preserved throughout a simulation trajectory).

In [3]:
from site_analysis.atom import atoms_from_species_string
atoms = atoms_from_species_string(structure, 'Na')
atoms[0:3]

[site_analysis.Atom(index=0, in_site=None, frac_coords=None),
 site_analysis.Atom(index=1, in_site=None, frac_coords=None),
 site_analysis.Atom(index=2, in_site=None, frac_coords=None)]

We also need to define our sites. In this example we want **polyhedral** sites, where each site is defined as the polyhedral volume enclosed by a set of vertex atoms. We therefore need to find the atom indices of the vertex atoms for each of our sites.

In [4]:
from pymatgen.io.vasp import Poscar, Xdatcar
import numpy as np
import operator
from site_analysis.polyhedral_site import PolyhedralSite
from site_analysis.atom import atoms_from_species_string
from site_analysis.trajectory import Trajectory
from site_analysis.tools import get_vertex_indices

To do this we load a `POSCAR` file where every octahedral site is occupied by a Na atom.

In [5]:
all_na_structure = Poscar.from_file('na_sn_all_na_ext.POSCAR.vasp').structure
vertex_species = 'S'
centre_species = 'Na'
all_na_structure.composition

Comp: Na136 Sn16 P8 S96

We then use the `get_vertex_indices()` function to find the six closest S to each Na (within a cutoff of 4.3 Å).  
This returns a nested list, where each sublist contains the S indices for a single polyhedron. In this case we have 136 Na atoms, but only 88 in our real simulation trajectory, so we subtract 48 to align the vertex indices.

In [6]:
# find atom indices (within species) for all polyhedra vertex atoms
vertex_indices = np.array(get_vertex_indices(all_na_structure, centre_species=centre_species, 
                                             vertex_species=vertex_species, cutoff=4.3))
print(vertex_indices[:4])
# Our real structures contain 88 Na atoms, so we correct these vertex indices
vertex_indices = vertex_indices - 48
print(vertex_indices[:4])

[[242 218 186 244 188 220]
 [250 252 178 212 180 210]
 [230 174 224 206 168 200]
 [166 192 232 208 182 249]]
[[194 170 138 196 140 172]
 [202 204 130 164 132 162]
 [182 126 176 158 120 152]
 [118 144 184 160 134 201]]


We can now use these vertex ids to define our `Polyhedron` objects.  

In [7]:
sites = PolyhedralSite.sites_from_vertex_indices(vertex_indices)

In [8]:
sites[:3]

[site_analysis.PolyhedralSite(index=0, label=None, vertex_indices=[194 170 138 196 140 172], contains_atoms=[]),
 site_analysis.PolyhedralSite(index=1, label=None, vertex_indices=[202 204 130 164 132 162], contains_atoms=[]),
 site_analysis.PolyhedralSite(index=2, label=None, vertex_indices=[182 126 176 158 120 152], contains_atoms=[])]

We now create a `Trajectory` object:

In [9]:
trajectory = Trajectory(sites=sites,
                        atoms=atoms)
trajectory

The `Trajectory` object provides the main interface for working with the `Polyhedron` and `Atom` objects.  

To analyse the site occupation for a particular `pymatgen` `Structure`:

In [10]:
trajectory.analyse_structure(structure)

The list of sites occupied by each atom can now be accessed using `trajectory.atom_sites`

In [11]:
np.array(trajectory.atom_sites)

array([  9,  92,  99,  14,  16,  68,  66,  23,  96,  83,  58,  56,  86,
        54,  64,  15,  65,  12,  91,  13,  94,  50,  60,  85,  53,  24,
        78,  82,  32,  34,   5,  40,   4,  42,   3,  35,  33,  41,   7,
        39,  11,  93,  49,  67,  26,  81,   6,  62,  69,  48,   0,  97,
        22,  90,  61,  55,   8,  30, 101,  72,  31,   1,  71,  98,   2,
        89,  21,  70,  87,  25,  84,  20,  46,  36,  80,  17,  51,  63,
        88,  44,  59,  52,  57,  43,  45, 100,  18,  79])

e.g. atom index 0 is occupying site index 9 (both counting from zero)

In [12]:
s = trajectory.sites[9]
print(s.contains_atom(trajectory.atoms[0]))
print(s.contains_atom(trajectory.atoms[1]))

True
False


In [13]:
s = trajectory.site_by_index(9)
print(s.contains_atom(trajectory.atom_by_index(0)))
print(s.contains_atom(trajectory.atom_by_index(1)))

True
False


The list of atoms occupying each site can be accessed using `trajectory.site_occupations`.  
The occupations of each site are stored as a list of lists, as each site can have zero, one, or multiple atoms occupying it.

In [14]:
trajectory.site_occupations

[[50],
 [61],
 [64],
 [34],
 [32],
 [30],
 [46],
 [38],
 [56],
 [0],
 [],
 [40],
 [17],
 [19],
 [3],
 [15],
 [4],
 [75],
 [86],
 [],
 [71],
 [66],
 [52],
 [7],
 [25],
 [69],
 [44],
 [],
 [],
 [],
 [57],
 [60],
 [28],
 [36],
 [29],
 [35],
 [73],
 [],
 [],
 [39],
 [31],
 [37],
 [33],
 [83],
 [79],
 [84],
 [72],
 [],
 [49],
 [42],
 [21],
 [76],
 [81],
 [24],
 [13],
 [55],
 [11],
 [82],
 [10],
 [80],
 [22],
 [54],
 [47],
 [77],
 [14],
 [16],
 [6],
 [43],
 [5],
 [48],
 [67],
 [62],
 [59],
 [],
 [],
 [],
 [],
 [],
 [26],
 [87],
 [74],
 [45],
 [27],
 [9],
 [70],
 [23],
 [12],
 [68],
 [78],
 [65],
 [53],
 [18],
 [1],
 [41],
 [20],
 [],
 [8],
 [51],
 [63],
 [2],
 [85],
 [58],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

A *trajectory* consists of a series of site occupations over multiple timesteps. A single timestep can be processed using the `analysis.append_timestep()` method:

In [15]:
trajectory.append_timestep(structure)

There are two ways to think about a site-projected trajectory:  
1. From an atom-centric perspective. Each atom visits a series of sites, and occupies one site each timestep.
2. From a site-centric perspective. Each site is visited by a series of atoms, and has zero, one, or more atoms occupying it at each timestep.

These two trajectory types can be accessed using the `Atom.trajectory` and `Site.trajectory` attributes:

In [16]:
print(atoms[3].trajectory)

[14]


In [17]:
print(sites[3].trajectory)

[[34]]


In [18]:
trajectory.timesteps

[None]

In [19]:
np.array(trajectory.atoms_trajectory)

array([[  9,  92,  99,  14,  16,  68,  66,  23,  96,  83,  58,  56,  86,
         54,  64,  15,  65,  12,  91,  13,  94,  50,  60,  85,  53,  24,
         78,  82,  32,  34,   5,  40,   4,  42,   3,  35,  33,  41,   7,
         39,  11,  93,  49,  67,  26,  81,   6,  62,  69,  48,   0,  97,
         22,  90,  61,  55,   8,  30, 101,  72,  31,   1,  71,  98,   2,
         89,  21,  70,  87,  25,  84,  20,  46,  36,  80,  17,  51,  63,
         88,  44,  59,  52,  57,  43,  45, 100,  18,  79]])

In [20]:
trajectory.sites_trajectory

[[[50],
  [61],
  [64],
  [34],
  [32],
  [30],
  [46],
  [38],
  [56],
  [0],
  [],
  [40],
  [17],
  [19],
  [3],
  [15],
  [4],
  [75],
  [86],
  [],
  [71],
  [66],
  [52],
  [7],
  [25],
  [69],
  [44],
  [],
  [],
  [],
  [57],
  [60],
  [28],
  [36],
  [29],
  [35],
  [73],
  [],
  [],
  [39],
  [31],
  [37],
  [33],
  [83],
  [79],
  [84],
  [72],
  [],
  [49],
  [42],
  [21],
  [76],
  [81],
  [24],
  [13],
  [55],
  [11],
  [82],
  [10],
  [80],
  [22],
  [54],
  [47],
  [77],
  [14],
  [16],
  [6],
  [43],
  [5],
  [48],
  [67],
  [62],
  [59],
  [],
  [],
  [],
  [],
  [],
  [26],
  [87],
  [74],
  [45],
  [27],
  [9],
  [70],
  [23],
  [12],
  [68],
  [78],
  [65],
  [53],
  [18],
  [1],
  [41],
  [20],
  [],
  [8],
  [51],
  [63],
  [2],
  [85],
  [58],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []]]

Example of processing a simulation trajectory using the `XDATCAR` file:  
(using `analysis.reset()` to reset the trajectory data.)

In [21]:
trajectory.reset()

xdatcar = Xdatcar('XDATCAR')

for timestep, s in enumerate(xdatcar.structures):
    trajectory.append_timestep(s, t=timestep)

Checking which sites has Na(3) visited:  

In [22]:
print(trajectory.atom_by_index(3).trajectory) # convert to a numpy array to then use numpy array slicing to extract a single atom trajectory.

[14, 14, 14, 14, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72]


Na(4) starts in site 14, and moves to site 72 at timestep 5.  
The same information can be seen by querying the site occupation data for sites 15 and 73:  
(note use of `analysis.st` as shorthand for `analysis.sites_trajectory`)

In [23]:
print(trajectory.site_by_index(14).trajectory)
print(trajectory.site_by_index(72).trajectory)

[[3], [3], [3], [3], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
[[], [], [], [], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3]]
